In [1]:
# Importing modules

import socks
import socket
import requests
import codecs
from bs4 import BeautifulSoup

## Data Extraction from HTML

In [18]:
file = codecs.open("../raw-html/lockbit.html", "r", "utf-8")
  
# Reading the file
index = file.read()
  
# Creating a BeautifulSoup object and specifying the parser
soup = BeautifulSoup(index, 'lxml')

In [ ]:
# Printing html code of body tag
print(soup.body.prettify())

## Data Cleaning

In [ ]:
# Obtaining all posts
ransomware_posts = soup.find_all("div", {"class": "post-block bad"})
print(ransomware_posts)

### Titles

In [ ]:
post_titles_tag = soup.find_all("div", {"class": "post-title"})
post_titles = [tag.text.strip() for tag in post_titles_tag]
print(post_titles)

### Descriptions

In [ ]:
post_desc_tag = soup.find_all("div", {"class": "post-block-text"})
post_desc = [tag.text.strip() for tag in post_desc_tag]
post_desc = [desc.replace('\r\n', ' ').replace('\r\n\r', ' ') for desc in post_desc]
print(post_desc)

### Dates

In [ ]:
post_date_tag = soup.find_all("div", {"class": "updated-post-date"})
post_date = [tag.text.strip() for tag in post_date_tag]
post_date = [tag.replace('\xa0\xa0', '').replace('Updated: ', '') for tag in post_date]
print(post_date)

### Views

In [ ]:
post_views_tag = soup.find_all("span", {"style": "font-size: 12px; font-weight: bold"})
post_views = [tag.text.strip() for tag in post_views_tag]
print(post_views)

In [45]:
# Verify all are extracted
print(len(post_titles) == len(post_desc) == len(post_date) == len(post_views))

True


In [66]:
import pandas as pd
import numpy as np

df = pd.DataFrame(columns=['Company Name', 'Location', 'Industry', 'Post Title', 'Description', 'Post Date', 'Post Views'])
df['Post Title'] = post_titles
df['Description'] = post_desc
df['Post Date'] = post_date
df['Post Views'] = post_views

df
df.to_csv('Lockbit2.csv')

In [5]:
import pandas as pd

df = pd.read_csv('../csv-data/Lockbit2.csv')
df.head()

,Unnamed: 0,Company Name,Location,Industry,Post Title,Description,Post Date,Post Views
0,0,NaN,NaN,NaN,parrottsims.com,We are a full-service law firm providing our c...,"24 Sep, 2022, 00:03 UTC",346
1,1,NaN,NaN,NaN,rbroof.com,Phone Number: (888) 260-3947 Owners phone num...,"23 Sep, 2022, 15:21 UTC",243
2,2,NaN,NaN,NaN,ch-sf.fr,We present to you the French clinic. CH-SF.FR ...,"23 Sep, 2022, 12:19 UTC",2134
3,3,NaN,NaN,NaN,congerbuilt.com,When Larry Conger founded Conger Construction ...,"22 Sep, 2022, 23:20 UTC",263
4,4,NaN,NaN,NaN,thorguard.com,THOR GUARD began manufacturing Lightning Predi...,"22 Sep, 2022, 23:05 UTC",274


## Selenium automation

In [17]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

### Searching data from Glassdoor

In [86]:
options = webdriver.ChromeOptions()
options.add_argument('--disable-blink-features=AutomationControlled') 
options.add_argument("enable-automation");
# options.add_argument("--headless");
options.add_argument("--no-sandbox");
options.add_argument("--disable-extensions");
options.add_argument("--dns-prefetch-disable");
options.add_argument("--disable-gpu");

path = 'D:/Downloads/chromedriver'
driver = webdriver.Chrome(options=options, executable_path=path)

for i in range(len(df)):
    try:
        company = df['Post Title'][i]
        driver.get('https://www.google.com')
        search = driver.find_element(By.NAME, 'q')
        search.send_keys(company + ' glassdoor "overview"')
        search.send_keys(Keys.RETURN)
        if i == 200: time.sleep(20)
        glassdoor_page = driver.find_element_by_tag_name('h3') # clicking the first search result
        glassdoor_page.click()
        
        if "glassdoor" not in driver.current_url:
            print('none')
            continue
        
        pagesource = driver.page_source
        soup = BeautifulSoup(pagesource, 'html.parser')
        company_name = company_location = company_industry = '-'
        
        name = soup.find(id='DivisionsDropdownComponent')
        company_name = name.text
        df['Company Name'][i] = company_name
        
        labels = soup.find_all('label')
        for label in labels:
            if label.text == "Headquarters:":
                company_location = label.next_sibling.text
                df['Location'][i] = company_location
            elif (label.text == "Industry:"):
                company_industry = label.next_sibling.text
                df['Industry'][i] = company_industry

        print(f'{i}: {company_name}, {company_location}, {company_industry}')
        
    except:
        print("none")
    
driver.quit()

D:\ProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\ProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\ProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


200:  Cabbage, Sheffield Village, OH, Wholesale
201:  A2 Consulting, -, -
202:  Genus plc, Basingstoke, United Kingdom, Biotech & Pharmaceuticals
203:  Vytelle, Hermiston, OR, -
none
205:  Export Trading Group (ETG), Dar es Salaam, Tanzania, Wholesale
206:  Lonseal, Carson, CA, Consumer Product Manufacturing
207:  Metro Appliances & More, Jonesboro, AR, Consumer Electronics & Appliances Stores
208:  Carn Brea Leisure Centre, Redruth, United Kingdom, -
209:  AXELLIUM, Rennes, France, -
210:  Sigma, San Pedro Garza García, Mexico, Food & Beverage Manufacturing
none
none
none
none


In [87]:
df.to_csv('lockbit_data.csv')

### Searching data from zoominfo

In [ ]:
for i in range(15, 20):
    try:        
        driver.get('https://www.google.com')
        header = df['Post Title'][i]
        
        search = driver.find_element(By.NAME, 'q')
        search.send_keys(header + ' zoominfo')
        search.send_keys(Keys.RETURN)
        glassdoor_page = driver.find_element_by_tag_name('h3') # clicking the first search result
        glassdoor_page.click()

        if "zoominfo" in driver.current_url:
            company = driver.find_element_by_tag_name('h1')
            company_name = company.text

            company_info = driver.find_element_by_class_name('company-header-subtitle')
            company_info = company_info.text
            company_info_list = company_info.split(' · ')

            company_industry = company_info_list[0]
            company_location = company_info_list[1]   

            df['Company Name'][i] = company_name
            df['Location'][i] = company_location
            df['Industry'][i] = company_industry

    except Exception as e:
        print(e)
    
driver.quit()
df.head(20)